In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
pip install numpy==1.26.4

: 

In [1]:
pip install tensorFlow==2.17.0

ERROR: Could not find a version that satisfies the requirement tensorFlow==2.17.0 (from versions: none)
ERROR: No matching distribution found for tensorFlow==2.17.0
Note: you may need to restart the kernel to use updated packages.


In [3]:
!pip install pyvi

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.5/8.5 MB 193.5 kB/s eta 0:00:0000:0100:02


In [ ]:
import numpy as np
import pandas as pd
from string import digits
import tensorflow as tf
from pyvi import ViTokenizer
%matplotlib inline

: 

In [2]:
!pip install gensim

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.6/60.6 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.7/26.7 MB 74.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.6/38.6 MB 23.6 MB/s eta 0:00:00
  Attempting uninstall: scipy
    Found existing installation: scipy 1.14.1
    Uninstalling scipy-1.14.1:
      Successfully uninstalled scipy-1.14.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tf-keras 2.18.0 requires tensorflow<2.19,>=2.18, but you have tensorflow 2.12.0 which is incompatible.
imbalanced-learn 0.13.0 requires numpy<3,>=1.24.3, but you have numpy 1.23.5 which is incompatible.
chex 0.1.89 requires numpy>=1.24.1, but you have numpy 1.23.5 which is incompatible.
pymc 5.21.1 requires numpy>=1.25.0, but you have numpy 1.23.5 which is incompatible.
albumentations 2.0.5 requires numpy>=1.24.4, but you have numpy

In [3]:
data_train = pd.read_csv("/content/drive/MyDrive/NLP/vlsp_sentiment_test.csv", sep='\t')
data_train.columns =['Class', 'Data']
data_test = pd.read_csv("/content/drive/MyDrive/NLP/vlsp_sentiment_train.csv", sep='\t')
data_test.columns =['Class', 'Data']

In [4]:
print(data_train.shape)
print(data_test.shape)

(1050, 2)
(5100, 2)


In [5]:
labels = data_train.iloc[:, 0].values
reviews = data_train.iloc[:, 1].values

In [6]:
encoded_labels = []

for label in labels:
    if label == -1:
        encoded_labels.append([1,0,0])
    elif label == 0:
        encoded_labels.append([0,1,0])
    else:
        encoded_labels.append([0,0,1])

encoded_labels = np.array(encoded_labels)

In [7]:
reviews_processed = []
unlabeled_processed = []
for review in reviews:
    review_cool_one = ''.join([char for char in review if char not in digits])
    reviews_processed.append(review_cool_one)

In [8]:
#Use PyVi for Vietnamese word tokenizer
word_reviews = []
all_words = []
for review in reviews_processed:
    review = ViTokenizer.tokenize(review.lower())
    word_reviews.append(review.split())


In [9]:
EMBEDDING_DIM = 400 # how big is each word vector
MAX_VOCAB_SIZE = 10000 # how many unique words to use (i.e num rows in embedding vector)
MAX_SEQUENCE_LENGTH = 300 # max number of words in a comment to use

In [10]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical

In [11]:
tokenizer = Tokenizer(num_words=MAX_VOCAB_SIZE, lower=True, char_level=False)
tokenizer.fit_on_texts(word_reviews)
sequences_train = tokenizer.texts_to_sequences(word_reviews)
word_index = tokenizer.word_index


In [12]:
data = pad_sequences(sequences_train, maxlen=MAX_SEQUENCE_LENGTH)
labels = encoded_labels

In [13]:
print('Shape of X train and X validation tensor:',data.shape)
print('Shape of label train and validation tensor:', labels.shape)

Shape of X train and X validation tensor: (1050, 300)
Shape of label train and validation tensor: (1050, 3)


In [14]:
import gensim
from gensim.models import Word2Vec
from gensim.utils import simple_preprocess

from gensim.models.keyedvectors import KeyedVectors

word_vectors = KeyedVectors.load_word2vec_format('/content/drive/MyDrive/NLP/vi-model-CBOW.bin', binary=True)

vocabulary_size=min(len(word_index)+1,MAX_VOCAB_SIZE)
embedding_matrix = np.zeros((vocabulary_size, EMBEDDING_DIM))
for word, i in word_index.items():
    if i>=MAX_VOCAB_SIZE:
        continue
    try:
        embedding_vector = word_vectors[word]
        embedding_matrix[i] = embedding_vector
    except KeyError:
        embedding_matrix[i]=np.random.normal(0,np.sqrt(0.25),EMBEDDING_DIM)

del(word_vectors)

from keras.layers import Embedding
embedding_layer = Embedding(vocabulary_size,
                            EMBEDDING_DIM,
                            weights=[embedding_matrix],
                            trainable=True)

In [15]:
from tensorflow.keras.layers import Dense, Input, GlobalMaxPooling1D, BatchNormalization, SeparableConv1D, Conv1D, MaxPooling1D, Embedding, Input, MaxPool1D, Dense, Embedding, Dropout,concatenate, Reshape, Flatten, Concatenate, Dropout, SpatialDropout1D
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import regularizers
import tensorflow as tf
from tensorflow.keras.layers import Dense, Input, Conv1D, GlobalMaxPooling1D, Embedding, Dropout, concatenate, Flatten
from tensorflow.keras.optimizers import SGD

In [16]:
sequence_length = data.shape[1]
filter_sizes = [3,4,5]
num_filters = 100
dropout_rate = 0.5
inputs = Input(shape=(sequence_length,))
embedding = embedding_layer(inputs)
num_classes = 3

# 1. Code gốc CNN

In [17]:
# reshape = Reshape((sequence_length,EMBEDDING_DIM,1))(embedding)
conv_0 = Conv1D(num_filters, filter_sizes[0],activation='relu',kernel_regularizer=regularizers.l2(0.01))(embedding)
conv_1 = Conv1D(num_filters, filter_sizes[1],activation='relu',kernel_regularizer=regularizers.l2(0.01))(embedding)
conv_2 = Conv1D(num_filters, filter_sizes[2],activation='relu',kernel_regularizer=regularizers.l2(0.01))(embedding)
print(conv_1)
maxpool_0 = MaxPooling1D(sequence_length - filter_sizes[0] + 1, strides=1)(conv_0)
maxpool_1 = MaxPooling1D(sequence_length - filter_sizes[1] + 1, strides=1)(conv_1)
maxpool_2 = MaxPooling1D(sequence_length - filter_sizes[2] + 1, strides=1)(conv_2)

merged_tensor = concatenate([maxpool_0, maxpool_1, maxpool_2], axis=1)
flatten = Flatten()(merged_tensor)
reshape = Reshape((3*num_filters,))(flatten)
dropout = Dropout(dropout_rate)(flatten)
output = Dense(units=num_classes, activation='softmax',kernel_regularizer=regularizers.l2(0.01))(dropout)

# this creates a model that includes
model_org = Model(inputs, output, name="CNN Org")

model_org.summary()
model_infor = {}
model_infor['CNN_Org'] = model_org

KerasTensor(type_spec=TensorSpec(shape=(None, 297, 100), dtype=tf.float32, name=None), name='conv1d_1/Relu:0', description="created by layer 'conv1d_1'")
Model: "CNN Org"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 300)]        0           []                               
                                                                                                  
 embedding (Embedding)          (None, 300, 400)     1386000     ['input_1[0][0]']                
                                                                                                  
 conv1d (Conv1D)                (None, 298, 100)     120100      ['embedding[0][0]']              
                                                                                                  
 conv1d_1 (Conv1D)              (None

# 2. Code cải thiện CNN + BiLSTM + Attention larer

In [19]:
from tensorflow.keras.layers import LSTM, SpatialDropout1D, Bidirectional, Dense, Dropout
filter_sizes = [2, 3, 4, 5]  # dùng nhiều filter hơn
drop = 0.4
inputs = Input(shape=(sequence_length,))
embedding = embedding_layer(inputs)
reshape = Reshape((sequence_length, EMBEDDING_DIM))(embedding)

# ===================== CNN Branch =====================
convs = []
for size in filter_sizes:
    conv = SeparableConv1D(filters=num_filters,
                           kernel_size=size,
                           activation='relu',
                           padding='same',
                           kernel_regularizer=regularizers.l2(0.01))(reshape)
    conv = BatchNormalization()(conv)
    conv = GlobalMaxPooling1D()(conv)
    convs.append(conv)

cnn_concat = concatenate(convs)

# ===================== BiLSTM Branch =====================
bilstm = Bidirectional(LSTM(128, return_sequences=True))(reshape)

# Attention Layer
def attention_layer(inputs):
    score = Dense(1, activation='tanh')(inputs)
    weights = tf.nn.softmax(score, axis=1)
    context = weights * inputs
    return tf.reduce_sum(context, axis=1)

attn_output = attention_layer(bilstm)

# ===================== Merge CNN + BiLSTM + Attention =====================
merged = concatenate([cnn_concat, attn_output])
merged = Dropout(drop)(merged)
output = Dense(3, activation='softmax', kernel_regularizer=regularizers.l2(0.01))(merged)

model_improve_cnn = Model(inputs, output, name="CRNN_Enhanced")

# Hiển thị kiến trúc mô hình
model_improve_cnn.summary()
model_infor['CRNN_Enhanced'] = model_improve_cnn

Model: "CRNN_Enhanced"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_3 (InputLayer)           [(None, 300)]        0           []                               
                                                                                                  
 embedding (Embedding)          (None, 300, 400)     1386000     ['input_3[0][0]']                
                                                                                                  
 reshape_2 (Reshape)            (None, 300, 400)     0           ['embedding[2][0]']              
                                                                                                  
 bidirectional (Bidirectional)  (None, 300, 256)     541696      ['reshape_2[0][0]']              
                                                                                      

# 3. LSTM

In [20]:
from tensorflow.keras.layers import LSTM, SpatialDropout1D, Dense, Dropout

embedding_dropout = SpatialDropout1D(0.2)(embedding)
lstm = LSTM(128, return_sequences=False, dropout=dropout_rate, recurrent_dropout=dropout_rate)(embedding_dropout)
dense = Dense(128, activation='relu')(lstm)
drop = Dropout(dropout_rate)(dense)
output = Dense(units=num_classes, activation='softmax')(drop)

model_lstm = Model(inputs, output, name="LSTM_Simple")
model_infor['LSTM_Simple'] = model_lstm

# Stacked LSTM (2 lớp nối tiếp)

In [21]:
from tensorflow.keras.layers import LSTM, SpatialDropout1D, Dense, Dropout

embedding_dropout = SpatialDropout1D(0.2)(embedding)
lstm_1 = LSTM(256, return_sequences=True, dropout=dropout_rate, recurrent_dropout=dropout_rate)(embedding_dropout)
lstm_2 = LSTM(128, return_sequences=False, dropout=dropout_rate, recurrent_dropout=dropout_rate)(lstm_1)
dense = Dense(128, activation='relu')(lstm_2)
drop = Dropout(dropout_rate)(dense)
output = Dense(units=num_classes, activation='softmax')(drop)

model_stacked_lstm = Model(inputs, output, name="Stacked_LSTM")
model_infor['Stacked_LSTM'] = model_stacked_lstm

# Song song BiLSTM đầu-cuối (Parallel BiLSTM)

In [22]:
from tensorflow.keras.layers import Bidirectional, LSTM, concatenate

embedding_dropout = SpatialDropout1D(0.2)(embedding)

# BiLSTM 1
bilstm1 = Bidirectional(LSTM(128, return_sequences=False, dropout=dropout_rate))(embedding_dropout)

# BiLSTM 2
bilstm2 = Bidirectional(LSTM(64, return_sequences=False, dropout=dropout_rate))(embedding_dropout)

# Kết hợp song song
merged = concatenate([bilstm1, bilstm2])
dense = Dense(128, activation='relu')(merged)
drop = Dropout(dropout_rate)(dense)
output = Dense(units=num_classes, activation='softmax')(drop)

model_parallel_bilstm = Model(inputs, output, name="Parallel_BiLSTM")
model_infor['Parallel_BiLSTM'] = model_parallel_bilstm


In [ ]:
CNN + LSTM kết hợp (CNN -> LSTM)

In [23]:
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Reshape

# CNN
reshape = Reshape((sequence_length, EMBEDDING_DIM))(embedding)
cnn = Conv1D(128, 5, activation='relu', padding='same')(reshape)
pool = MaxPooling1D(pool_size=2)(cnn)

# LSTM
lstm = LSTM(128, return_sequences=False, dropout=dropout_rate, recurrent_dropout=dropout_rate)(pool)

# Dense layers
dense = Dense(128, activation='relu')(lstm)
drop = Dropout(dropout_rate)(dense)
output = Dense(units=num_classes, activation='softmax')(drop)

model_cnn_lstm = Model(inputs, output, name="CNN_LSTM")
model_infor['CNN_LSTM'] = model_cnn_lstm

# 4. CNN +BiLSTM

In [24]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Embedding, Conv1D, GlobalMaxPooling1D, concatenate
from tensorflow.keras.layers import Bidirectional, LSTM, Dropout, Dense, BatchNormalization

# Multi-kernel CNN
conv_0 = Conv1D(filters=128,
                kernel_size=3,
                padding='same',
                activation='relu',
                kernel_regularizer=regularizers.l2(0.01))(embedding)
conv_1 = Conv1D(filters=128, kernel_size=4, padding='same', activation='relu', kernel_regularizer=regularizers.l2(0.01))(embedding)
conv_2 = Conv1D(filters=128, kernel_size=5, padding='same', activation='relu',kernel_regularizer=regularizers.l2(0.01))(embedding)

# MaxPooling
maxpool_0 = GlobalMaxPooling1D()(conv_0)
maxpool_1 = GlobalMaxPooling1D()(conv_1)
maxpool_2 = GlobalMaxPooling1D()(conv_2)

# BiLSTM
bilstm = Bidirectional(LSTM(128, return_sequences=False))(embedding)

# Kết hợp BiLSTM và CNN
merged = concatenate([bilstm, maxpool_0, maxpool_1, maxpool_2])

# Thêm BatchNormalization và Dropout
x = BatchNormalization()(merged)
x = Dropout(dropout_rate)(x)

# Dense layer trung gian giúp học tốt hơn
x = Dense(128, activation='relu')(x)
x = Dropout(dropout_rate)(x)

# Output
output = Dense(units=3, activation='softmax')(x)

# Model
model_cnn_bilstm = Model(inputs, output, name="CNN_BiLSTM")
model_cnn_bilstm.summary()
model_infor['CNN_BiLSTM'] = model_cnn_bilstm

Model: "CNN_BiLSTM"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_3 (InputLayer)           [(None, 300)]        0           []                               
                                                                                                  
 embedding (Embedding)          (None, 300, 400)     1386000     ['input_3[0][0]']                
                                                                                                  
 conv1d_4 (Conv1D)              (None, 300, 128)     153728      ['embedding[2][0]']              
                                                                                                  
 conv1d_5 (Conv1D)              (None, 300, 128)     204928      ['embedding[2][0]']              
                                                                                         

# 5. LSTMs + CNN

In [25]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Embedding, Conv1D, GlobalMaxPooling1D, concatenate
from tensorflow.keras.layers import Bidirectional, LSTM, Dropout, Dense, BatchNormalization
from tensorflow.keras import regularizers

def build_lstm_cnn_model(inputs,
                         embedding,
                         lstm_units=128,
                         cnn_filters=128,
                         dropout_rate=0.5,
                         num_classes=3):
    """
    Xây dựng mô hình với kiến trúc Embedding -> BiLSTM -> CNN.

    Args:

    Returns:
        tensorflow.keras.models.Model: Mô hình Keras.
    """


    # BiLSTM
    bilstm = Bidirectional(LSTM(lstm_units, return_sequences=True))(embedding) # return_sequences=True cho CNN sau đó

    # Multi-kernel CNN
    conv_0 = Conv1D(filters=cnn_filters,
                    kernel_size=3,
                    padding='same',
                    activation='relu',
                    kernel_regularizer=regularizers.l2(0.01))(bilstm)
    conv_1 = Conv1D(filters=cnn_filters,
                    kernel_size=4, padding='same',
                    activation='relu',
                    kernel_regularizer=regularizers.l2(0.01))(bilstm)
    conv_2 = Conv1D(filters=cnn_filters, kernel_size=5,
                    padding='same',
                    activation='relu',
                    kernel_regularizer=regularizers.l2(0.01))(bilstm)

    # MaxPooling
    maxpool_0 = GlobalMaxPooling1D()(conv_0)
    maxpool_1 = GlobalMaxPooling1D()(conv_1)
    maxpool_2 = GlobalMaxPooling1D()(conv_2)

    # Kết hợp BiLSTM và CNN
    merged = concatenate([maxpool_0, maxpool_1, maxpool_2]) # BiLSTM đã được xử lý bởi CNN

    # Thêm BatchNormalization và Dropout
    x = BatchNormalization()(merged)
    x = Dropout(dropout_rate)(x)

    x = Dense(lstm_units, activation='relu')(x) # Giữ số units tương đương LSTM
    x = Dropout(dropout_rate)(x)

    # Output
    output = Dense(units=num_classes, activation='softmax')(x)

    # Model
    model_lstm_cnn = Model(inputs, output, name="LSTM_CNN")
    return model_lstm_cnn

# Tạo và tóm tắt mô hình
model_lstm_cnn = build_lstm_cnn_model(
    inputs,
    embedding,
    dropout_rate=dropout_rate,
    num_classes=3 # Thay đổi nếu số lượng lớp của bạn khác
)
model_lstm_cnn.summary()

# Lưu thông tin mô hình (nếu bạn đang sử dụng dictionary model_infor)
if 'model_infor' in locals():
    model_infor['LSTM_CNN'] = model_lstm_cnn

Model: "LSTM_CNN"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_3 (InputLayer)           [(None, 300)]        0           []                               
                                                                                                  
 embedding (Embedding)          (None, 300, 400)     1386000     ['input_3[0][0]']                
                                                                                                  
 bidirectional_4 (Bidirectional  (None, 300, 256)    541696      ['embedding[2][0]']              
 )                                                                                                
                                                                                                  
 conv1d_7 (Conv1D)              (None, 300, 128)     98432       ['bidirectional_4[0][0]'] 

# 6. CRNN code gốc của thầy

In [26]:
sequence_length = data.shape[1]
filter_sizes = [3,4,5]
num_filters = 100
drop = 0.5

inputs = Input(shape=(sequence_length,))
embedding = embedding_layer(inputs)

################## LSTM ONLY ###############################
# reshape = Reshape((sequence_length,EMBEDDING_DIM))(embedding)

################# SINGLE LSTM ####################
# lstm_0 = LSTM(512)(reshape)

# YOU WANNA ADD MORE LSTM LAYERS? UNCOMMENT THIS #
# lstm_2 = LSTM(1024, return_sequences=True)(reshape)
# lstm_1 = LSTM(512, return_sequences=True)(lstm_2)
# lstm_0 = LSTM(256)(lstm_1)

############################################################


################## CRNN ####################################
reshape = Reshape((sequence_length,EMBEDDING_DIM))(embedding)

conv_0 = Conv1D(num_filters,
 (filter_sizes[0], ),
                padding="same",
                activation='relu',
                kernel_regularizer=regularizers.l2(0.01))(reshape)
conv_1 = Conv1D(num_filters,
 (filter_sizes[1], ),
                padding="same",
                activation='relu',
                kernel_regularizer=regularizers.l2(0.01))(reshape)
conv_2 = Conv1D(num_filters,
 (filter_sizes[2], ),
                padding="same",
                activation='relu',
                kernel_regularizer=regularizers.l2(0.01))(reshape)

conv_0 = MaxPool1D(300)(conv_0)
conv_1 = MaxPool1D(300)(conv_1)
conv_2 = MaxPool1D(300)(conv_2)
# Reshape output to match RNN dimension
# conv_0 = Reshape((-1, num_filters))(conv_0)
# conv_1 = Reshape((-1, num_filters))(conv_1)
# conv_2 = Reshape((-1, num_filters))(conv_2)

concat = concatenate([conv_0, conv_1, conv_2])
concat = Flatten()(concat)
# lstm_0 = LSTM(512)(concat)

# YOU WANNA ADD MORE LSTM LAYERS? UNCOMMENT THIS #
# lstm_2 = LSTM(1024, return_sequences=True)(concat)
# lstm_1 = LSTM(512, return_sequences=True)(lstm_2)
# lstm_0 = LSTM(256)(lstm_1)

############################################################

dropout = Dropout(drop)(concat)
output = Dense(units=3, activation='softmax',kernel_regularizer=regularizers.l2(0.01))(dropout)
# this creates a model that includes
model_crnn = Model(inputs, output, name="CRNN")
model_crnn.summary()
model_infor['CRNN'] = model_crnn

Model: "CRNN"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_4 (InputLayer)           [(None, 300)]        0           []                               
                                                                                                  
 embedding (Embedding)          (None, 300, 400)     1386000     ['input_4[0][0]']                
                                                                                                  
 reshape_4 (Reshape)            (None, 300, 400)     0           ['embedding[3][0]']              
                                                                                                  
 conv1d_10 (Conv1D)             (None, 300, 100)     120100      ['reshape_4[0][0]']              
                                                                                               

# CRNN (CNN + BiLSTM)

In [27]:
from tensorflow.keras.layers import Bidirectional, LSTM

reshape = Reshape((sequence_length, EMBEDDING_DIM))(embedding)

# CNN với nhiều filter size
conv_blocks = []
for size in filter_sizes:
    conv = Conv1D(num_filters, kernel_size=size, padding="same", activation='relu',
                  kernel_regularizer=regularizers.l2(0.01))(reshape)
    conv = MaxPool1D(pool_size=2)(conv)
    conv_blocks.append(conv)

cnn_concat = concatenate(conv_blocks)

# BiLSTM sau CNN
bilstm = Bidirectional(LSTM(128, return_sequences=False))(cnn_concat)

dense = Dense(128, activation='relu')(bilstm)
drop = Dropout(drop)(dense)
output = Dense(units=3, activation='softmax', kernel_regularizer=regularizers.l2(0.01))(drop)

model_crnn_improved = Model(inputs, output, name="CRNN_Improved")
model_infor['CRNN_Improved'] = model_crnn_improved

# RNN + CNN + LSTM (theo thứ tự RNN → CNN → LSTM)

In [28]:
from tensorflow.keras.layers import SimpleRNN

reshape = Reshape((sequence_length, EMBEDDING_DIM))(embedding)

# RNN đơn trước
rnn = SimpleRNN(128, return_sequences=True)(reshape)

# CNN tiếp theo
conv = Conv1D(num_filters, 5, activation='relu', padding='same')(rnn)
pool = MaxPool1D(pool_size=2)(conv)

# LSTM cuối cùng
lstm = LSTM(128, return_sequences=False)(pool)

dense = Dense(128, activation='relu')(lstm)
drop = Dropout(drop)(dense)
output = Dense(units=3, activation='softmax')(drop)

model_rnn_cnn_lstm = Model(inputs, output, name="RNN_CNN_LSTM")
model_infor['RNN_CNN_LSTM'] = model_rnn_cnn_lstm

# LSTM + RNN song song (LSTM & RNN chạy song song rồi nối lại)

In [29]:
reshape = Reshape((sequence_length, EMBEDDING_DIM))(embedding)

# LSTM
lstm_branch = LSTM(128, return_sequences=False)(reshape)

# RNN
rnn_branch = SimpleRNN(128, return_sequences=False)(reshape)

# Ghép song song
merged = concatenate([lstm_branch, rnn_branch])
dense = Dense(128, activation='relu')(merged)
drop = Dropout(drop)(dense)
output = Dense(units=3, activation='softmax')(drop)

model_lstm_rnn = Model(inputs, output, name="LSTM_RNN_Parallel")
model_infor['LSTM_RNN_Parallel'] = model_lstm_rnn

# 7. BERT-based Approach

In [30]:
# -*- coding: utf-8 -*-
import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    AdamW,
    get_linear_schedule_with_warmup
)
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, classification_report, confusion_matrix
import time
import datetime
import random
import numpy as np
import matplotlib.pyplot as plt
import os # Thêm import os

# --- 1. Configuration ---
# <<< THAY ĐỔI CÁC THAM SỐ NÀY NẾU CẦN >>>
MODEL_NAME = 'vinai/phobert-base' # Hoặc 'vinai/phobert-large' nếu có GPU mạnh
MAX_LEN = 128      # Độ dài tối đa sequence (phù hợp với PhoBERT)
BATCH_SIZE = 16     # Giảm nếu gặp lỗi Out-of-Memory (OOM) trên GPU
EPOCHS = 4          # Số epochs huấn luyện (thường 2-5 là đủ cho fine-tuning)
LEARNING_RATE = 2e-5 # Learning rate phổ biến cho fine-tuning BERT
NUM_LABELS = 3      # Số lượng nhãn sentiment (VD: 0: Tiêu cực, 1: Trung tính, 2: Tích cực)
RANDOM_SEED = 42
OUTPUT_DIR = './bert_sentiment_model/' # Thư mục lưu model tốt nhất

# --- Đường dẫn dữ liệu (thay thế bằng đường dẫn thực tế) ---
# Giả sử file CSV có cột 'text' và 'label' (label là số nguyên 0, 1, 2...)
TRAIN_DATA_PATH = 'path/to/your/sentiment_data.csv' # <<< THAY ĐỔI ĐƯỜNG DẪN NÀY
TEXT_COLUMN = 'text'
LABEL_COLUMN = 'label'
# TEST_DATA_PATH = 'path/to/your/test.csv' # Nếu có file test riêng

# --- Thiết lập seed để có thể tái tạo kết quả ---
random.seed(RANDOM_SEED)
np.random.seed(RANDOM_SEED)
torch.manual_seed(RANDOM_SEED)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(RANDOM_SEED)
    # Cấu hình thêm để đảm bảo tính xác định (có thể làm chậm quá trình huấn luyện)
    # torch.backends.cudnn.deterministic = True
    # torch.backends.cudnn.benchmark = False

# --- Thiết lập device (GPU nếu có) ---
if torch.cuda.is_available():
    device = torch.device("cuda")
    print(f'Using GPU: {torch.cuda.get_device_name(0)}')
else:
    device = torch.device("cpu")
    print('Using CPU')

# --- 2. Load Data ---
print(f"\nLoading data from: {TRAIN_DATA_PATH}")
try:
    df = pd.read_csv(TRAIN_DATA_PATH)
    print("Data loaded successfully.")
    print(f"Dataset shape: {df.shape}")
    print("\nSample data:")
    print(df.head())
    print(f"\nValue counts for labels ({LABEL_COLUMN}):")
    print(df[LABEL_COLUMN].value_counts())

    # Kiểm tra số lượng nhãn thực tế
    actual_num_labels = df[LABEL_COLUMN].nunique()
    if actual_num_labels != NUM_LABELS:
        print(f"\n*** Warning: NUM_LABELS is set to {NUM_LABELS}, but found {actual_num_labels} unique labels in the data. Please check your configuration and data.")
        # NUM_LABELS = actual_num_labels # Cập nhật nếu muốn tự động điều chỉnh

    # Đảm bảo cột text là string và xử lý NaN
    df[TEXT_COLUMN] = df[TEXT_COLUMN].astype(str).fillna('')
    df = df[[TEXT_COLUMN, LABEL_COLUMN]] # Chỉ giữ lại cột cần thiết

except FileNotFoundError:
    print(f"Error: File not found at {TRAIN_DATA_PATH}")
    exit()
except KeyError as e:
    print(f"Error: Column '{e}' not found in the CSV file. Ensure your CSV has columns named '{TEXT_COLUMN}' and '{LABEL_COLUMN}'.")
    exit()
except Exception as e:
    print(f"An unexpected error occurred during data loading: {e}")
    exit()


# Chia dữ liệu thành train và validation (chưa dùng test set ở đây)
print("\nSplitting data into training and validation sets...")
df_train, df_val = train_test_split(
    df,
    test_size=0.1, # 10% cho validation
    random_state=RANDOM_SEED,
    stratify=df[LABEL_COLUMN] # Giữ tỷ lệ nhãn trong cả 2 tập
)

print(f"Train size: {len(df_train)}")
print(f"Validation size: {len(df_val)}")
# print(f"Test size: {len(df_test)}") # Nếu có

# --- 3. Tokenizer & Dataset ---
print(f"\nLoading Tokenizer for {MODEL_NAME}...")
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

class SentimentDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_len):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, item):
        text = str(self.texts[item])
        label = self.labels[item]

        encoding = self.tokenizer.encode_plus(
            text,
            add_special_tokens=True, # Thêm '[CLS]' và '[SEP]' (hoặc '<s>', '</s>' cho PhoBERT)
            max_length=self.max_len,
            return_token_type_ids=False, # Không cần cho sequence classification với 1 câu
            padding='max_length', # Pad đến max_length
            truncation=True,      # Cắt nếu dài hơn max_length
            return_attention_mask=True,
            return_tensors='pt',  # Trả về PyTorch tensors
        )

        return {
            'text': text,
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'labels': torch.tensor(label, dtype=torch.long)
        }

def create_data_loader(df, tokenizer, max_len, batch_size, num_workers=2):
    ds = SentimentDataset(
        texts=df[TEXT_COLUMN].to_numpy(),
        labels=df[LABEL_COLUMN].to_numpy(),
        tokenizer=tokenizer,
        max_len=max_len
    )
    return DataLoader(
        ds,
        batch_size=batch_size,
        num_workers=num_workers, # Số worker để load data song song
        shuffle=True # Shuffle dữ liệu trong quá trình huấn luyện
    )

# Tạo DataLoader cho train và validation
train_data_loader = create_data_loader(df_train, tokenizer, MAX_LEN, BATCH_SIZE)
# Không cần shuffle validation loader
val_dataset = SentimentDataset(
    texts=df_val[TEXT_COLUMN].to_numpy(),
    labels=df_val[LABEL_COLUMN].to_numpy(),
    tokenizer=tokenizer,
    max_len=MAX_LEN
)
val_data_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, num_workers=2, shuffle=False)

# test_data_loader = create_data_loader(df_test, tokenizer, MAX_LEN, BATCH_SIZE) # Nếu có test set

# --- 4. Model ---
print(f"\nLoading Pre-trained Model: {MODEL_NAME} for Sequence Classification...")
model = AutoModelForSequenceClassification.from_pretrained(
    MODEL_NAME,
    num_labels=NUM_LABELS,
    output_attentions=False, # Không cần attention weights khi fine-tuning
    output_hidden_states=False, # Không cần hidden states khi fine-tuning
)
model.to(device)
print("Model loaded and moved to device.")

# --- 5. Optimizer & Scheduler ---
# AdamW là optimizer thường dùng cho BERT
optimizer = torch.optim.AdamW(model.parameters(), lr=LEARNING_RATE, eps=1e-8)

total_steps = len(train_data_loader) * EPOCHS # Tổng số bước huấn luyện

# Scheduler để giảm learning rate theo thời gian
scheduler = get_linear_schedule_with_warmup(
    optimizer,
    num_warmup_steps=0, # Số bước "khởi động" (thường là 0 hoặc ~10% total_steps)
    num_training_steps=total_steps
)

# --- 6. Training & Evaluation Functions ---
def format_time(elapsed):
    '''Chuyển đổi thời gian (giây) sang định dạng hh:mm:ss'''
    elapsed_rounded = int(round((elapsed)))
    return str(datetime.timedelta(seconds=elapsed_rounded))

def train_epoch(model, data_loader, optimizer, device, scheduler, epoch_num, total_epochs):
    """Huấn luyện một epoch."""
    print(f'\n======== Epoch {epoch_num + 1} / {total_epochs} ========')
    print('Training...')
    t0 = time.time()
    total_loss = 0
    model.train() # Chuyển model sang chế độ training

    predictions_train, true_labels_train = [], []

    for step, batch in enumerate(data_loader):
        # Log tiến trình mỗi 50 steps
        if step % 50 == 0 and not step == 0:
            elapsed = format_time(time.time() - t0)
            print(f'  Batch {step:>5,}  of  {len(data_loader):>5,}.    Elapsed: {elapsed}.')

        # Đưa batch lên device (GPU/CPU)
        b_input_ids = batch['input_ids'].to(device)
        b_attention_mask = batch['attention_mask'].to(device)
        b_labels = batch['labels'].to(device)

        model.zero_grad() # Xóa gradient từ bước trước

        # Forward pass
        outputs = model(
            input_ids=b_input_ids,
            token_type_ids=None, # BERT không cần cái này cho single sequence tasks
            attention_mask=b_attention_mask,
            labels=b_labels # Model tự tính loss khi có labels
        )

        loss = outputs.loss
        logits = outputs.logits
        total_loss += loss.item()

        # Backward pass để tính gradient
        loss.backward()

        # Chống gradient exploding (gradient clipping)
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        # Cập nhật weights
        optimizer.step()
        # Cập nhật learning rate
        scheduler.step()

        # Lưu lại dự đoán và nhãn thật để tính accuracy/f1 cuối epoch
        logits = logits.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()
        predictions_train.extend(np.argmax(logits, axis=1).flatten())
        true_labels_train.extend(label_ids.flatten())


    avg_train_loss = total_loss / len(data_loader)
    training_time = format_time(time.time() - t0)

    # Tính accuracy và F1 score cho epoch này
    train_accuracy = accuracy_score(true_labels_train, predictions_train)
    train_f1 = f1_score(true_labels_train, predictions_train, average='weighted') # 'weighted' cho multi-class

    print(f"\n  Average training loss: {avg_train_loss:.4f}")
    print(f"  Training Accuracy: {train_accuracy:.4f}")
    print(f"  Training F1 Score: {train_f1:.4f}")
    print(f"  Training epoch took: {training_time}")

    return avg_train_loss, train_accuracy, train_f1


def eval_model(model, data_loader, device):
    """Đánh giá model trên tập dữ liệu (validation hoặc test)."""
    print("\nRunning Validation...")
    t0 = time.time()
    model.eval() # Chuyển model sang chế độ evaluation

    total_eval_loss = 0
    predictions, true_labels = [], []

    with torch.no_grad(): # Không cần tính gradient khi đánh giá
        for batch in data_loader:
            b_input_ids = batch['input_ids'].to(device)
            b_attention_mask = batch['attention_mask'].to(device)
            b_labels = batch['labels'].to(device)

            outputs = model(
                input_ids=b_input_ids,
                token_type_ids=None,
                attention_mask=b_attention_mask,
                labels=b_labels
            )

            loss = outputs.loss
            logits = outputs.logits

            total_eval_loss += loss.item()

            # Chuyển logits và labels về CPU để tính metrics
            logits = logits.detach().cpu().numpy()
            label_ids = b_labels.to('cpu').numpy()

            predictions.extend(np.argmax(logits, axis=1).flatten())
            true_labels.extend(label_ids.flatten())

    avg_val_loss = total_eval_loss / len(data_loader)
    validation_time = format_time(time.time() - t0)

    # Tính accuracy và F1 score
    val_accuracy = accuracy_score(true_labels, predictions)
    val_f1 = f1_score(true_labels, predictions, average='weighted') # Hoặc 'macro'

    print(f"\n  Validation Loss: {avg_val_loss:.4f}")
    print(f"  Validation Accuracy: {val_accuracy:.4f}")
    print(f"  Validation F1 Score: {val_f1:.4f}")
    print(f"  Validation took: {validation_time}")

    # In báo cáo chi tiết hơn
    print("\nClassification Report (Validation):")
    target_names = [f'Label_{i}' for i in range(NUM_LABELS)] # Đặt tên nhãn nếu biết
    print(classification_report(true_labels, predictions, target_names=target_names, digits=4))

    print("\nConfusion Matrix (Validation):")
    print(confusion_matrix(true_labels, predictions))

    return avg_val_loss, val_accuracy, val_f1


# --- 7. Training Loop ---
history = {'train_loss': [], 'train_acc': [], 'train_f1': [],
           'val_loss': [], 'val_acc': [], 'val_f1': []}
best_val_accuracy = 0
total_t0 = time.time()

print("\n--- Starting Training ---")
for epoch_i in range(0, EPOCHS):

    # --- Training ---
    avg_train_loss, train_accuracy, train_f1 = train_epoch(
        model, train_data_loader, optimizer, device, scheduler, epoch_i, EPOCHS
    )

    # --- Validation ---
    avg_val_loss, val_accuracy, val_f1 = eval_model(
        model, val_data_loader, device
    )

    # Lưu lại lịch sử
    history['train_loss'].append(avg_train_loss)
    history['train_acc'].append(train_accuracy)
    history['train_f1'].append(train_f1)
    history['val_loss'].append(avg_val_loss)
    history['val_acc'].append(val_accuracy)
    history['val_f1'].append(val_f1)

    # Lưu model tốt nhất dựa trên validation accuracy
    if val_accuracy > best_val_accuracy:
        best_val_accuracy = val_accuracy
        print(f"\n  Saving best model to {OUTPUT_DIR} (Epoch {epoch_i + 1})")

        # Tạo thư mục nếu chưa có
        if not os.path.exists(OUTPUT_DIR):
            os.makedirs(OUTPUT_DIR)

        # Lưu model và tokenizer
        model_to_save = model.module if hasattr(model, 'module') else model # Xử lý khi dùng DataParallel
        model_to_save.save_pretrained(OUTPUT_DIR)
        tokenizer.save_pretrained(OUTPUT_DIR)
        # Có thể lưu thêm các tham số khác nếu cần
        # torch.save(args, os.path.join(output_dir, 'training_args.bin'))

print("\n--- Training complete! ---")
print(f"Total training took {format_time(time.time()-total_t0)}")
print(f"Best validation accuracy achieved: {best_val_accuracy:.4f}")


# --- 9. Plot learning curves ---
def plot_bert_history(history, model_name="BERT"):
    """Vẽ đồ thị accuracy, F1 và loss trong quá trình huấn luyện BERT."""
    epochs_range = range(1, len(history['train_acc']) + 1)

    plt.figure(figsize=(18, 5))

    # Accuracy plot
    plt.subplot(1, 3, 1)
    plt.plot(epochs_range, history['train_acc'], label='Training Accuracy')
    plt.plot(epochs_range, history['val_acc'], label='Validation Accuracy')
    plt.legend(loc='lower right')
    plt.title(f'{model_name} - Accuracy')
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    plt.grid(True)

    # F1 Score plot
    plt.subplot(1, 3, 2)
    plt.plot(epochs_range, history['train_f1'], label='Training F1 Score')
    plt.plot(epochs_range, history['val_f1'], label='Validation F1 Score')
    plt.legend(loc='lower right')
    plt.title(f'{model_name} - F1 Score (Weighted)')
    plt.xlabel('Epochs')
    plt.ylabel('F1 Score')
    plt.grid(True)

    # Loss plot
    plt.subplot(1, 3, 3)
    plt.plot(epochs_range, history['train_loss'], label='Training Loss')
    plt.plot(epochs_range, history['val_loss'], label='Validation Loss')
    plt.legend(loc='upper right')
    plt.title(f'{model_name} - Loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.grid(True)

    plt.suptitle(f'Learning Curves for {model_name}', fontsize=16)
    plt.tight_layout(rect=[0, 0.03, 1, 0.95]) # Adjust layout
    plt.show()

print("\nPlotting learning curves...")
plot_bert_history(history, model_name=MODEL_NAME)

ImportError: cannot import name 'AdamW' from 'transformers' (/usr/local/lib/python3.11/dist-packages/transformers/__init__.py)

# 4. Training model

In [31]:
import tensorflow as tf
tf.config.run_functions_eagerly(True)


In [ ]:
# Compile models
for name, model in model_infor.items():
    optimizer = Adam(learning_rate=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-8)
    model.compile(loss='categorical_crossentropy',
                  optimizer=optimizer,
                  metrics=['accuracy'])

# Train models
history = {}
for name, model in model_infor.items():
    # Define callbacks
    early_stopping = EarlyStopping(monitor='val_loss', patience=4, verbose=1)
    callbacks_list = [early_stopping]
    print(f"Training model '{name}'...")
    history[name] = model.fit(data, labels,
                               validation_split=0.2,
                               epochs=5,
                               batch_size=256,
                               callbacks=callbacks_list,
                               shuffle=True)

Training model 'CNN_Org'...
Epoch 1/5
4/4 [==============================] - 29s 6s/step - loss: 4.8017 - accuracy: 0.8083 - val_loss: 6.6625 - val_accuracy: 0.1048
Epoch 2/5
4/4 [==============================] - 18s 4s/step - loss: 4.4529 - accuracy: 0.8726 - val_loss: 7.5935 - val_accuracy: 0.0714
Epoch 3/5
4/4 [==============================] - 19s 4s/step - loss: 4.1265 - accuracy: 0.9119 - val_loss: 6.4488 - val_accuracy: 0.1238
Epoch 4/5
4/4 [==============================] - 19s 4s/step - loss: 3.9652 - accuracy: 0.9298 - val_loss: 6.6837 - val_accuracy: 0.1190
Epoch 5/5
4/4 [==============================] - 17s 4s/step - loss: 3.7657 - accuracy: 0.9357 - val_loss: 6.9113 - val_accuracy: 0.0524
Training model 'CRNN_Enhanced'...
Epoch 1/5
4/4 [==============================] - 57s 14s/step - loss: 4.0792 - accuracy: 0.3988 - val_loss: 1.4148 - val_accuracy: 0.0000e+00
Epoch 2/5
4/4 [==============================] - 54s 13s/step - loss: 3.0562 - accuracy: 0.4405 - val_loss: 1.4

ERROR:tensorflow:==================================
Object was never used (type <class 'tensorflow.python.ops.tensor_array_ops.TensorArray'>):
If you want to mark it as used call its "mark_used()" method.
It was originally created here:
  File "/usr/local/lib/python3.11/dist-packages/keras/backend.py", line 5159, in <genexpr>
    output_ta_t = tuple(  File "/usr/local/lib/python3.11/dist-packages/tensorflow/python/util/tf_should_use.py", line 243, in wrapped
    return _add_should_use_warning(fn(*args, **kwargs),


4/4 [==============================] - 70s 19s/step - loss: 2.2163 - accuracy: 0.5690 - val_loss: 1.5458 - val_accuracy: 0.0000e+00
Epoch 4/5
4/4 [==============================] - 51s 13s/step - loss: 1.8432 - accuracy: 0.6095 - val_loss: 1.3278 - val_accuracy: 0.0000e+00
Epoch 5/5
4/4 [==============================] - 52s 13s/step - loss: 1.4466 - accuracy: 0.6714 - val_loss: 1.3457 - val_accuracy: 0.0000e+00
Training model 'LSTM_Simple'...
Epoch 1/5
4/4 [==============================] - 49s 12s/step - loss: 1.1000 - accuracy: 0.3619 - val_loss: 1.5697 - val_accuracy: 0.0000e+00
Epoch 2/5
4/4 [==============================] - 50s 12s/step - loss: 1.0495 - accuracy: 0.4429 - val_loss: 1.8151 - val_accuracy: 0.0000e+00
Epoch 3/5
4/4 [==============================] - 49s 12s/step - loss: 1.0171 - accuracy: 0.4810 - val_loss: 1.9706 - val_accuracy: 0.0000e+00
Epoch 4/5
4/4 [==============================] - 56s 14s/step - loss: 1.0299 - accuracy: 0.4405 - val_loss: 2.0208 - val_accur

# Đánh giá

In [ ]:
labels_test = data_test.iloc[:, 0].values
reviews_test = data_test.iloc[:, 1].values

In [ ]:
encoded_labels_test = []

for label_test in labels_test:
    if label_test == -1:
        encoded_labels_test.append([1,0,0])
    elif label_test == 0:
        encoded_labels_test.append([0,1,0])
    else:
        encoded_labels_test.append([0,0,1])

encoded_labels_test = np.array(encoded_labels_test)

In [ ]:
reviews_processed_test = []
unlabeled_processed_test = []
for review_test in reviews_test:
    review_cool_one = ''.join([char for char in review_test if char not in digits])
    reviews_processed_test.append(review_cool_one)

In [ ]:
#Use PyVi for Vietnamese word tokenizer
word_reviews_test = []
all_words = []
for review_test in reviews_processed_test:
    review_test = ViTokenizer.tokenize(review_test.lower())
    word_reviews_test.append(review_test.split())

In [ ]:
sequences_test = tokenizer.texts_to_sequences(word_reviews_test)
data_test = pad_sequences(sequences_test, maxlen=MAX_SEQUENCE_LENGTH)
labels_test = encoded_labels_test

In [ ]:
print('Shape of X train and X validation tensor:',data_test.shape)
print('Shape of label train and validation tensor:', labels_test.shape)

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

def compare_models_performance(history_dict, model_dict, X_test, y_test):
    """
    So sánh hiệu suất của các model bao gồm:
    - Validation accuracy, validation loss theo epochs
    - Accuracy, loss, precision, recall trên tập test
    - Các biểu đồ trực quan hóa

    Args:
        history_dict: dict chứa lịch sử huấn luyện (tên -> history)
        model_dict: dict chứa models (tên -> model)
        X_test: dữ liệu kiểm thử
        y_test: nhãn kiểm thử
    """
    # 3. Đánh giá test set
    test_metrics = {}

    print("\nMODEL TEST PERFORMANCE")
    print("=" * 50)
    for name, model in model_dict.items():
        results = model.evaluate(X_test, y_test, verbose=0)
        metrics_names = model.metrics_names
        test_metrics[name] = dict(zip(metrics_names, results))

        metric_str = ' | '.join([
            f"{k.capitalize()}: {v*100:.2f}%" if k != 'loss' else f"Loss: {v:.4f}"
            for k, v in test_metrics[name].items()
        ])
        print(f"{name:20s} -> {metric_str}")

    # 4. Bar chart: Test Accuracy (%)
    plt.figure(figsize=(10, 5))
    acc_values = [test_metrics[name].get('accuracy', 0) * 100 for name in model_dict.keys()]
    plt.bar(model_dict.keys(), acc_values, color='skyblue')
    plt.title('Test Accuracy Comparison')
    plt.ylabel('Accuracy (%)')
    plt.grid(axis='y')
    for i, acc in enumerate(acc_values):
        plt.text(i, acc / 2, f'{acc:.2f}%', ha='center', color='black', fontweight='bold')
    plt.show()

    # 5. Bar chart: Test Loss
    plt.figure(figsize=(10, 5))
    loss_values = [test_metrics[name].get('loss', 0) for name in model_dict.keys()]
    plt.bar(model_dict.keys(), loss_values, color='salmon')
    plt.title('Test Loss Comparison')
    plt.ylabel('Loss %')
    plt.grid(axis='y')
    for i, loss in enumerate(loss_values):
        plt.text(i, loss / 2, f'{loss:.2f}%', ha='center', color='black', fontweight='bold')
    plt.show()

    return test_metrics

test_accuracies = compare_models_performance(history, model_infor, data_test, labels_test)

In [ ]:
for name, model in model_infor.items():
  print(name)
  score = model.evaluate( data_test, labels_test)
  print("%s: %.2f%%" % (model.metrics_names[0], score[0]*100))
  print("%s: %.2f%%" % (model.metrics_names[1], score[1]*100))
